# Fraud Detection Project

## Previous Notebooks

- [EDA](1-EDA.ipynb)
- [Network Analysis](2.1-Network.ipynb)
- [Lawyers' Network Analysis](2.2-Network-Lawyers.ipynb)

In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In this notebook I will propagate the PageRank calculated for the entities involved in a claim to all the witnesses involved in a claim; then again I will use this scores to calculate an overall score for a claim.

To propagate the score I will create a network of all witnesses and entities, using claims as links, and calculate PageRank on the resulting graph assigning the previously calculated weights to the entities and 0 to the witnesses as a starting score.

In [2]:
network = pd.read_csv('../data/raw/network_witnesses.csv', sep=';')

In [3]:
nw = nx.MultiGraph()
nw.add_edges_from(network[['id_lt', 'id_rt']].values);

In [4]:
print('Number of nodes: {}\nNumber of edges: {}\nNumber of connected components: {}'
      .format(nw.number_of_nodes(), nw.number_of_edges(), nx.number_connected_components(nw)))

Number of nodes: 17340
Number of edges: 13584
Number of connected components: 5285


In [5]:
node_scores = pd.read_pickle('../data/interim/node_scores.pkl')

In [6]:
weights_df = pd.DataFrame(list(nw.nodes())).merge(node_scores, how='left', left_on=0, right_index=True)\
                .drop('weights', axis=1).rename(columns={0:'id'})

In [7]:
weights_df['pagerank'].sum()

0.08020313547003893

I'm normalizing the weights to make them sum to 1, as it is requested by PageRank:

In [8]:
weights = (weights_df['pagerank'] / weights_df['pagerank'].sum()).fillna(0).values
weights.sum()

1.0000000000000007

## PageRank

In [9]:
from scipy import sparse
from sklearn.preprocessing import normalize

In [10]:
adj = nx.adj_matrix(nw, weight=None)
M = normalize(adj, norm='l1', axis=1)

In [11]:
v = weights.reshape(-1, 1)

In [12]:
# initializing the rank with 1/number of nodes
r = np.ones(len(v)) / len(v)
r = r.reshape(-1, 1)

In [13]:
# precision level
eps = 1e-9
d = 0.9 # damping factor
r_prev = r
r = d * M.T.dot(r_prev) + (1-d) * v
while np.linalg.norm(r - r_prev) > eps:
    r_prev = r
    r = d * M.T.dot(r_prev) + (1-d) * v

In [14]:
node_scores = pd.DataFrame(np.hstack([v, r]), index=list(weights_df['id']))
node_scores.rename(columns={0: 'weights', 1:'pagerank'}, inplace=True)

In [15]:
node_scores.loc[network['id_rt'].unique()].sort_values(by='pagerank', ascending=False).head(25)

,weights,pagerank
210886,0.0,0.030113
215156,0.0,0.000769
214757,0.0,0.000609
212747,0.0,0.000523
215234,0.0,0.000507
211003,0.0,0.000435
214851,0.0,0.000416
214402,0.0,0.000416
210753,0.0,0.000397
214892,0.0,0.000385


## Scoring the Claims

As previously I'm going to use the mean of the scores of all the witnesses involved.

In [16]:
network = network.merge(node_scores, how='left', left_on='id_rt', right_index=True)\
                .drop('weights', axis=1).rename(columns={'pagerank':'witness_pagerank'})

In [17]:
network['witness_score'] = network['witness_pagerank']

In [18]:
nw_scores = network.groupby('claim_code')['witness_score'].mean().reset_index()
nw_scores.to_pickle('../data/interim/witnesses_network_scores.pkl')

## Following Notebooks

- [Dataset Creation](3-Input_dataset_creation.ipynb)
- [Random Forest Prediction](4-Model.ipynb)